# Set-up

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/2302_happyTEXT

/content/drive/MyDrive/2302_happyTEXT


In [ ]:
%%capture
! pip install --upgrade pip
! pip install openai
! pip install -U sentence-transformers
! pip install pyvis

In [6]:
import os
import math
import pandas as pd

from collections import deque

import nltk
nltk.download('wordnet')
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tag  import pos_tag

from sklearn.feature_extraction.text import TfidfVectorizer
import networkx as nx

from prompt_generator import ChatPromptGenerator
from heuristics import *
from utils import *
from graph import *

import openai

from tqdm import tqdm
import time
import re

import json
import csv
import pickle

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


ModuleNotFoundError: No module named 'openai'

In [ ]:
### Variables to receive as arguments
# How many events to generate
MAX_DEPTH = 20

# If True use few shot, elif False use zero shot
FEW_SHOT = False

# If True use reason precondition always
HAVE_REASON = True

# if True use precondition to make context in prompt.
CONTEXT_USED_PC = False

# API Key
GPT_API = 'gpt-3.5-turbo-16k-0613'
OPENAI_API_KEY = 'your key'

# 0: None, 1: preconditions & events, 2: prompts, 3: Error
DEBUG = 2

# Need to fix when converting to NSP.py
NOTEBOOK = False

# For turbo, system content
#system_content = "You are the story planner. Imagine the conditions that will satisfy the event and the event that will satisfy the condition. If it doesn't make sense, answer with 'None.'"
#system_content = "You are the most interesting story writer in the world. Now, before writing the entire story, you will create the plot of the story. If you give a simple context and ask a question, imagine what might happen and answer it briefly. If it doesn't make sense, answer with 'None'. Please answer juct one sentence."
system_content = "You are the most interesting story writer in the world. Now, before writing the entire story, you will create the plot of the story. If you give an (event) and ask a question, you can imagine the preconditions that will satisfy the event. If you give a (precondition) and ask a question, you can imagine an event that satisfies the precondition. If there is a precondition already satisfied in the conversation, tell me the same precondition. If it doesn't make sense, answer with 'None'. Please answer juct one sentence."

In [ ]:
model = GPT(GPT_API, api_key=OPENAI_API_KEY, system_content=system_content)
sbert = SentenceBert()

In [ ]:
# float values mean the order, lower is earlier plot
defined_pc_types = {'item_need': 3,
                    'location': 1,
                    'reason': 5,
                    # 'item_state': 2,
                    # 'how': 4,
                    # 'interaction': 0,
                    }

# Method

In [ ]:
def precondition_generation(g, model, event, debug=0, few_shot=True, prompt_generator=None):
    if prompt_generator is None:
        prompt_generator = PromptGenerator(g, event, context_used_pc=CONTEXT_USED_PC)
    new_preconditions = []
    depth = event.depth

    for pc_type in defined_pc_types:
        if pc_type == 'item_state' and event.item is None:
            continue
        if pc_type == 'interaction' and len(event.person) == 1:
            continue

        if prompt_generator is None:
            prompt = prompt_generator.generate(prompt_type=pc_type, few_shot=few_shot)
        else:
            prompt = prompt_generator.generate(pc_type, g, event)
        generated = model.generate(prompt, debug)
        filtered = pc_filter(pc_type, sentence=generated)

        if filtered is not None:
            if prompt_generator is not None:
                prompt_generator.addMessage("assistant", filtered)

            level = get_pc_node_level(defined_pc_types[pc_type])
            if pc_type == 'location' and len(event.person) > 1:
                for person in event.person:
                    new_precondition = Precondition(pc_type, [person], filtered, depth, level)
                    new_preconditions.append(new_precondition)
            else:
                new_precondition = Precondition(pc_type, event.person, filtered, depth, level)
                new_preconditions.append(new_precondition)
        else:
            if prompt_generator is not None:
                prompt_generator.pop()

        if debug > 1:
            print(f'Preconditon ({pc_type}): {filtered}\n')

    return new_preconditions

In [ ]:
def event_generation(g, model, context_event, precondition, debug=0, few_shot=True, prompt_generator=None):
    if precondition.pc_type in ('how', 'interaction', 'reason'):
        generated = precondition.sentence
    else:
        if prompt_generator is None:
            prompt_generator = PromptGenerator(g, context_event, sentence=precondition.sentence, context_used_pc=CONTEXT_USED_PC)
            prompt = prompt_generator.generate(prompt_type=f'satisfy_{precondition.pc_type}', few_shot=few_shot)
        else:
            prompt = prompt_generator.generate(f'satisfy_{precondition.pc_type}', g, context_event, sentence=precondition.sentence)
        generated = model.generate(prompt, debug)

        if prompt_generator is not None:
            prompt_generator.addMessage("assistant", generated)

    depth = precondition.depth + 1
    new_event = Event(precondition.person, generated, precondition.sentence, precondition.pc_type, depth)

    if debug > 1:
        print(f'Event (satisfy_{precondition.pc_type}): {generated}\n')

    return new_event

# Make Plan

In [ ]:
def plan_generation(model, sbert, goal, initial_conditions, max_depth, few_shot, have_reason, debug):
    g = PlanGraph(notebook=NOTEBOOK, layout=True)
    g.add_node(goal, color='blue', title='Goal')

    cpg = ChatPromptGenerator()

    planner_queue = deque()
    planner_queue.append(goal)
    while planner_queue:
        cur_event = planner_queue.popleft()
        new_preconditions = precondition_generation(g, model, cur_event, debug=debug, few_shot=few_shot, prompt_generator=cpg)
        print(cur_event.action)
        print('\n--- new_preconditions')

        if not have_reason:
            has_il = False
            for p in new_preconditions:
                if p.pc_type in {'item_need', 'location'}:
                    has_il = True
                if p.pc_type == 'reason' and has_il:
                    new_preconditions.remove(p)

        # reason event에서 location precon 나올 때, context 때문에 앞 선 이벤트의 location이 나옴; 을 해결하기 위함
        new_preconditions = [p for p in new_preconditions if p.pc_type != 'location' or cur_event.e_type != 'reason']

        valid_new_preconditions = []
        for p in new_preconditions:
            print(f'- [{p.pc_type}]: {p.sentence}')

            # Dangling precondition;
            # (1) condition meets initial conditions
            initial, sim = find_similar_precondition(sbert, p, initial_conditions)
            if initial is not None:
                initial_conditions.remove(initial)
                initial.level = 0.5
                initial.depth = cur_event.depth
                g.add_node(initial, color='red', title='Inintial Condition')
                g.add_edge(initial, cur_event)
                print(f'Dangling (initial: {initial.sentence}) ({p.sentence}: {sim:.6f})')
                continue

            # Reuse precondition;
            # if there exists a precondition c' ∈ Λ that is similar to c
            reuse, sim = find_similar_precondition(sbert, p, g.precondition_nodes)
            if reuse is not None:
                reuse.depth = cur_event.depth
                if g.add_edge(reuse, cur_event):
                    g.reuse_node(reuse)
                    print(f'Reuse condition: {reuse.sentence} ({p.sentence}: {sim:.6f})')
                continue

            valid_new_preconditions.append(p)
            #g.add_node(p, color='pink')
            g.add_node(p, color='#a6a4a4')
            if g.add_edge(p, cur_event):
                print('Valid')

        # find new event
        print('\n--- new_events')
        for p in valid_new_preconditions:
            new_event = event_generation(g, model, cur_event, p, debug=debug, few_shot=few_shot, prompt_generator=cpg)
            #g.add_node(new_event, color='skyblue')
            g.add_node(new_event, color='#262626')
            g.add_edge(new_event, p)

            print(f'- [{p.sentence}]: {new_event.action}')

            if new_event.depth < max_depth and new_event.action !='None':
                planner_queue.append(new_event)

        print('\n-------------------------------------------------------------------------\n')
    return g

In [ ]:
action = 'Jill bought the puppy and brought it home to her family.' #@param {type:"string"}
person = 'Jill ' #@param {type:"string"}
person = person[:-1]
print(person)
goals = []
goals.append(Event(person=[person], action=action, item=None))
initial_conditions_list = []
initial_conditions_list.append([Precondition('item_need', [person], 'a car', 0, 3)])

Jill


In [ ]:
action = 'Amy poured herself a large glass of Sherry and forgot about the flies.' #@param {type:"string"}
person = 'Amy ' #@param {type:"string"}
person = person[:-1]
print(person)
goals.append(Event(person=[person], action=action, item=None))
initial_conditions_list.append([Precondition('item_need', [person], 'a car', 0, 3)])

Amy


In [ ]:
action = 'Milo felt guilty and went home without taking anything.' #@param {type:"string"}
person = 'Milo ' #@param {type:"string"}
person = person[:-1]
print(person)
goals.append(Event(person=[person], action=action, item=None))
initial_conditions_list.append([Precondition('item_need', [person], 'a car', 0, 3)])

Milo


In [ ]:
action = 'Bob sold off their belongings and used the money to buy a nice car.' #@param {type:"string"}
person = 'Bob ' #@param {type:"string"}
person = person[:-1]
print(person)
goals.append(Event(person=[person], action=action, item=None))
initial_conditions_list.append([Precondition('item_need', [person], 'a car', 0, 3)])

Bob


In [ ]:
action = 'Leo became very successful and famous for his work.' #@param {type:"string"}
person = 'Leo ' #@param {type:"string"}
person = person[:-1]
print(person)
goals.append(Event(person=[person], action=action, item=None))
initial_conditions_list.append([Precondition('item_need', [person], 'a car', 0, 3)])

Leo


In [ ]:
plans = []
stories = []

results_path_long = 'results/long (>=15)'
results_path_short = 'results/short (<15)'

i = 0
for goal in goals:
    # plan = plan_generation(model, sbert, goal, initial_conditions, MAX_DEPTH, FEW_SHOT, HAVE_REASON, DEBUG)
    # story = plan.dfs_traversal(goal.action, reverse_dircetion=True)

    # for s in reversed(story):
    #         if not s.startswith('('):
    #             print(s)

    # plans.append(plan)
    # stories.append(story)
    # print('\n')
    initial_conditions = initial_conditions_list[i]
    i = i + 1
    plan = []
    story = []
    for i in range(10): # 스토리 길이 15이상 10번 해서 안나오면 그냥 진행
        plan = plan_generation(model, sbert, goal, initial_conditions, MAX_DEPTH, FEW_SHOT, HAVE_REASON, DEBUG)
        story = plan.dfs_traversal(goal.action, reverse_dircetion=True)
        if len(story) >= 15:
            break

    plot = []
    for s in reversed(story):
            if not s.startswith('('):
                print(s)
                plot.append(s)

    plans.append(plan)
    stories.append(story)
    print('\n')

    if len(plot) >= 15:
        save_plan(results_path_long, plan, goal, story)
    else:
        save_plan(results_path_short, plan, goal, story)

Streaming output truncated to the last 5000 lines.
{'role': 'user', 'content': '(precondition) Leo drove\nImagine that How did Leo get to Leo is in a vehicle on the road?'}
{'role': 'assistant', 'content': 'Leo obtained access to a vehicle'}
{'role': 'user', 'content': '(event) Leo sorted through his belongings\nWhat item does Leo have?'}
{'role': 'assistant', 'content': 'sorted belongings'}
{'role': 'user', 'content': '(event) Leo sorted through his belongings\nWhere is Leo?'}
{'role': 'assistant', 'content': 'Leo is in a space dedicated to organizing his belongings'}
{'role': 'user', 'content': '(precondition) Leo sorted through his belongings\nImagine that How did Leo get sorted belongings?'}
{'role': 'assistant', 'content': 'Leo went through each item'}
{'role': 'user', 'content': '(precondition) Leo sorted through his belongings\nImagine that How did Leo get to Leo is in a space dedicated to organizing his belongings?'}
{'role': 'assistant', 'content': 'Leo set aside time'}
{'role

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Cycle found

--- new_events

-------------------------------------------------------------------------

Leo was exposed to a wide range of activities
Leo pursued different hobbies
Leo discovered his passion through exploring different interests
Leo is passionate about his work
Leo faced
Leo developed resilience through overcoming challenges
Leo was raised in an environment that emphasized the value of hard work
Leo realized that running away from challenges wouldn't lead to growth
Leo believes that challenges are opportunities for personal
Leo embraced challenges as opportunities for growth
Leo is devoted to continuous improvement
Leo found a peaceful environment to focus on his tasks
Leo dedicated time
Leo organized his space to create a specific area for focused practice
Leo set up a designated area for focused practice
Leo practiced
Leo adopted effective task management strategies
Leo learned to prioritize his tasks
Leo
Leo believes in the power of self-improvement
Leo created a cal

# Save

In [ ]:
def save_plan(results_path, plan, goal, story):
    plot = []
    for s in reversed(story):
            if not s.startswith('('):
                plot.append(s)

    if not os.path.exists(results_path):
        os.mkdir(results_path)

    if not os.path.exists(f'{results_path}/{goal.action} ({len(plot)})'):
        os.mkdir(f'{results_path}/{goal.action} ({len(plot)})')

    plan.g.save_graph(name=f'{results_path}/{goal.action} ({len(plot)})/graph_tree.html')
    plan.g.options.layout.hierarchical = plan.g.options.layout.Hierarchical(enabled=False)
    plan.g.save_graph(name=f'{results_path}/{goal.action} ({len(plot)})/graph.html')
    with open(f'{results_path}/{goal.action} ({len(plot)})/ordered_plot.txt', 'w') as f:
        for s in reversed(story):
            if not s.startswith('('):
                f.write(s + '\n')
    with open(f'{results_path}/{goal.action} ({len(plot)})/adj_list.pickle', 'wb') as f:
        pickle.dump(plan.g.get_adj_list(), f)
    with open(f'{results_path}/{goal.action} ({len(plot)})/network_data.pickle', 'wb') as f:
        pickle.dump(plan.g.get_network_data(), f)

In [ ]:
results_path = 'results/myResult_chat'
results_path

'results/myResult_chat'

In [ ]:
for plan, goal, story in zip(plans, goals, stories):
    save_plan(results_path, plan, goal, story)

# Make & Save using dataset

In [ ]:
# Load sample_ROCStories dataset
df = pd.read_csv("sample_ROCStories_characterWANT.csv")
goals = []
initial_conditions_list = []

i = 0
for _, row in df.iterrows():
    # if i < 7:
    #     i += 1
    #     continue
    goals.append(Event(person=[row['person']], action=row['sentence5'], item=None))
    initial_conditions_list.append([Precondition('item_need', [row['person']], 'a car', 0, 3)])
    print(row['person'], ')', row['sentence5'])
    # if row['item'] == 'None':
    #     goals.append(Event(person=[row['person']], action=row['sentence5'], item=None))
    # else:
    #     goals.append(Event(person=[row['person']], action=row['sentence5'], item=row['item']))
initial_conditions = []

Tom ) Tom sat on his couch filled with regret about his actions.
Anita ) Anita melted the chocolate down and mixed it into the cake batter.
Kelly ) Kelly sat the repaired vase onto the table as if nothing happened.
Sonia ) Sonia finally managed to find one and bought a nice dress.
Bill ) Bill calls triple a and waits for help.
Kira ) Kira made pizzas for dinner.
Sarah ) Sarah continued cutting the watermelon.
Shane ) Shane stole over 4 million dollars from his employer.
Jake ) Jake threw a snowball at the wall.
Kate ) Kate ran in her new shoes.
James ) James finally purchased a wedding ring.
Sally ) Sally knocked over all the bowling pins at the rock bowling alley.
Ann ) Ann bought baby clothes at the department store.
Julie ) Julie bought the latest GPU to play high-end games.
Tom ) Tom created his own work to be displayed in a contemporary art exhibition.
Jun ) Jun wanted to drink water
Alice ) Alice left the house to buy a mystery novel.
Ark ) Ark found the lost key in the warehouse

In [ ]:
# 생성-저장 같이
plans = []
stories = []

results_path_long = 'results/long (>=15)'
results_path_short = 'results/short (<15)'

# if not os.path.exists(results_path_long):
#     os.mkdir(results_path)

i = 0
for goal in goals:
    initial_conditions = initial_conditions_list[i]

    plan = []
    story = []
    for i in range(10): # 스토리 길이 15이상 10번 해서 안나오면 그냥 진행
        plan = plan_generation(model, sbert, goal, initial_conditions, MAX_DEPTH, FEW_SHOT, HAVE_REASON, DEBUG)
        story = plan.dfs_traversal(goal.action, reverse_dircetion=True)
        if len(story) >= 15:
            break

    for s in reversed(story):
            if not s.startswith('('):
                print(s)

    plans.append(plan)
    stories.append(story)
    print('\n')

    if len(story) >= 15:
        save_plan(results_path_long, plan, goal, story)
    else:
        save_plan(results_path_short, plan, goal, story)

    i = i + 1

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
{'role': 'assistant', 'content': 'Susan came across the business opportunity through a referral'}
{'role': 'user', 'content': "(event) Susan's car broke down due to a manufacturing defect\nWhat item does Susan have?"}
{'role': 'assistant', 'content': 'broken car'}
{'role': 'user', 'content': "(event) Susan's car broke down due to a manufacturing defect\nWhere is Susan?"}
{'role': 'assistant', 'content': 'Susan is wherever her broken down car is located'}
{'role': 'user', 'content': "(event) Susan's car broke down due to a manufacturing defect\nImagine that Why Susan do that?"}
{'role': 'assistant', 'content': "Susan's car broke down unexpectedly due to a defect in its manufacturing process"}
{'role': 'user', 'content': "(precondition) Susan's car broke down due to a manufacturing defect\nImagine that How did Susan get to Susan is wherever her broken down car is located?"}
{'role': 'assistant', 'content': 'Susan called a towing service to transport he

# Load

In [ ]:
def load_plan(path):
    plot = []
    with open(f'{path}/ordered_plot.txt', 'r') as f:
        plot += f.read().strip('\n').split('\n')

    adj_list = {}
    with open(f'{path}/adj_list.pickle', 'rb') as f:
        adj_list = pickle.load(f)

    network_data = {}
    with open(f'{path}/network_data.pickle', 'rb') as f:
        network_data = pickle.load(f)

    return plot, adj_list, network_data

In [ ]:
goal_name = 'James was glad to be playing a sport at school. (32)'  #@param {type:'string'}
results_path = f'results/fabula_/{goal_name}'
results_path

'results/fabula_/James was glad to be playing a sport at school. (32)'

In [ ]:
plot, adj_list, network_data = load_plan(results_path)
plot

['James owns a car for transportation convenience',
 'it was vital for the success of his project',
 'James prepared the business documents himself by compiling to ensure accuracy',
 'James went to different locations to gather the various materials',
 'James collected the various materials from different sources',
 'James brought the necessary materials with him when he traveled to the location',
 'James traveled to the location where he needed to use the necessary materials',
 'James brought all the necessary materials',
 'James prepared the business documents himself by compiling',
 "It was a crucial business meeting that James couldn't afford to miss",
 'James received the appointment details via email to stay informed',
 'James drove his car to the store where he bought the smartphone',
 'James bought the smartphone from a store',
 'James accessed his email using his smartphone to stay connected',
 'James carried his smartphone with him wherever he went',
 'James accessed his emai

# Selection



In [ ]:
def make_skeleton_simple(plot, adj_list, network_data, k=5):
    except_list = ['None', '(reason) None', '(location) None', '(item_need) None']

    # fabula
    fabula = [x for x in plot if x.strip() not in except_list]

    # seletion score 초기화
    selection_score = {}
    for key, value in zip(fabula, [0]*len(fabula)):
        selection_score[key] = value

    # 각 rink 수 반영
    for edge in network_data[1]:
        for event in selection_score:
            if event in [edge["from"], edge["to"]]:
                selection_score[event] += 1

    # goal과의 거리로 나눔
    for key in selection_score:
        distance, path = search_path_in_adj_list(adj_list, key, fabula[-1], except_list)
        if distance == 0:
            selection_score[key] = 0
        else:
            selection_score[key] = selection_score[key] / math.sqrt(distance)

    # selection score 높은 이벤트 선택 (top k)
    top_k_selection = sorted(selection_score.items(), key=lambda item: item[1], reverse=True)[:k]   # [(node, score),...]
    top_k_selection = [x[0] for x in top_k_selection]

    # 선택된 이벤트와 goal까지 path
    selected_node_list = [fabula[-1]]
    for start_node in top_k_selection:
        _, path = search_path_in_adj_list(adj_list, start_node, fabula[-1], except_list)
        selected_node_list = selected_node_list + path
    selected_node_list = list(set(selected_node_list))

    # fabula에서 선택된 것 추출 (skeleton)
    skeleton = [x for x in fabula if x in selected_node_list]

    return skeleton

In [ ]:
def make_skeleton(plot, adj_list, network_data, k=5):
    except_list = ['None', '(reason) None', '(location) None', '(item_need) None']

    # fabula
    fabula = [x for x in plot if x.strip() not in except_list]

    # seletion score 초기화
    selection_score = {}
    for key, value in zip(fabula, [0]*len(fabula)):
        selection_score[key] = value

    # 각 rink 수 반영
    for edge in network_data[1]:
        for event in selection_score:
            if event in [edge["from"], edge["to"]]:
                selection_score[event] += 1

    # interest score 반영
    for key in selection_score:
        interest_score = int(get_interest_score(key, plot, adj_list, network_data))
        selection_score[key] += interest_score
        print(key, interest_score)

    # goal과의 거리로 나눔
    for key in selection_score:
        distance, path = search_path_in_adj_list(adj_list, key, fabula[-1], except_list)
        if distance == 0:
            selection_score[key] = 0
        else:
            selection_score[key] = selection_score[key] / math.sqrt(distance)

    # selection score 높은 이벤트 선택 (top k)
    top_k_selection = sorted(selection_score.items(), key=lambda item: item[1], reverse=True)[:k]   # [(node, score),...]
    top_k_selection = [x[0] for x in top_k_selection]

    # 선택된 이벤트와 goal까지 path
    selected_node_list = [fabula[-1]]
    for start_node in top_k_selection:
        _, path = search_path_in_adj_list(adj_list, start_node, fabula[-1], except_list)
        selected_node_list = selected_node_list + path
    selected_node_list = list(set(selected_node_list))

    # fabula에서 선택된 것 추출 (skeleton)
    skeleton = [x for x in fabula if x in selected_node_list]

    return skeleton

In [ ]:
def get_interest_score(event, plot, adj_list, network_data, model='gpt-3.5-turbo-16k-0613', max_tokens=1024, temperature=1.0, top_p=1, n=1, presence_penalty=1.0, frequency_penalty=1.0, api_key=None, debug=0):
    system_content = 'You create a skeleton story by selecting events from the tree-structured story planner. You have to look at the story planner given an adjacency list and assign interest score to events in event list. Please assign an appropriate interest score to event considering the fun of the event, causal rink, goal sentence, etc.'

    prompt = f'goal: {plot[-1]}'
    prompt += '\n\nadjacency list'
    for fr, to in adj_list.items():
        prompt += f'\n{fr} : {to}'
    prompt += '\n\nevent list'
    for e in plot[:-1]:
        prompt += f'\n{e}'
    prompt += f"\n\nPlease answer the interest score of the event '{event}' as an integer number between 0 and 9 points."

    if debug > 0:
        print(f'prompt:\n{prompt}')

    received = False
    score = ''
    while not score:
        try:
            score = openai.ChatCompletion.create(
                model=model,
                max_tokens=max_tokens,
                temperature=temperature,
                top_p=top_p,
                n=n,
                presence_penalty=presence_penalty,
                frequency_penalty=frequency_penalty,
                messages = [{"role": "system", "content": system_content}, {"role": "user", "content": prompt}]
            )['choices'][0]['message']['content']
            if any(char.isdigit() for char in score):
                received = True
        except Exception as e:
            if debug > 2:
                print('Error in API:', e)
            time.sleep(1)

    if debug > 0:
        print(f'response:\n{score}')

    return re.sub(r'\D', '', score)

In [ ]:
skeleton_simple = make_skeleton_simple(plot, adj_list, network_data, 9)
skeleton_simple

['Ludo was not feeling well for a long time',
 "Ludo's manager",
 "Ludo's work was taking a toll on his health",
 "Ludo got prescribed medicine from a pharmacist after presenting his doctor's prescription",
 'Ludo went to see a doctor for his illness',
 'Ludo have been suffering from an illness',
 'Ludo was feeling weak',
 'Ludo was recovering from an illness',
 'Ludo applied for a credit card at his bank',
 'Ludo drove back home from his workplace to purchase the subscription online',
 'Ludo purchased a subscription online using his credit card',
 'Ludo watched a lot of movies on the subscription during the next week.']

In [ ]:
skeleton = make_skeleton(plot, adj_list, network_data, 9)
skeleton

Ludo was not feeling well for a long time 7
Ludo was given a new project that he had to complete urgently 7
Ludo got the laptop from his company for work purposes 7
Ludo was trying to impress his boss by working hard 7
Ludo was working long hours without taking enough breaks 5
Ludo drove to his workplace in his car 6
Ludo got the documents from his supervisor 4
Ludo's manager called him for a meeting 7
Ludo received his employment contract from the HR department of his company after getting hired 7
Ludo's manager 7
Ludo's work was taking a toll on his health 7
Ludo drove himself to the clinic/hospital 7
Ludo got a prescription for the medicine from his doctor 8
Ludo drove his car to the pharmacy 7
Ludo got prescribed medicine from a pharmacist after presenting his doctor's prescription 8
Ludo went to see a doctor for his illness 8
Ludo have been suffering from an illness 7
Ludo was feeling weak 8
Ludo was recovering from an illness 7
Ludo planned to spend more time at home 6
Ludo drove

['Ludo went to see a doctor for his illness',
 'Ludo have been suffering from an illness',
 'Ludo was feeling weak',
 'Ludo was recovering from an illness',
 'Ludo planned to spend more time at home',
 'Ludo drove to the bank in his car',
 'Ludo applied for a credit card at his bank',
 'Ludo drove back home from his workplace to purchase the subscription online',
 'Ludo purchased a subscription online using his credit card',
 'Ludo watched a lot of movies on the subscription during the next week.']

In [ ]:
# save
if not os.path.exists(results_path):
        os.mkdir(results_path)

with open(f'{results_path}/skeleton.txt', 'w') as f:
    for s in skeleton:
        f.write(s + '\n')

# Skeleton Selection

## event-driven

https://wooono.tistory.com/285
먼가 이상함

https://stackoverflow.com/questions/57572184/how-to-apply-tfidf-on-test-set
이게 좀 더 맞는듯

암튼 다른 스토리 데이터셋으로 idf 학습해서 pickle로 들고오자


### Load train dataset

In [ ]:
story_dataset = []

In [ ]:
# CSV 파일 경로
DATASET_PATH = 'ROCStories_.csv'

# CSV 파일 열기
with open(DATASET_PATH, mode='r') as file:
    csv_reader = csv.reader(file)

    # 각 행에 대해서 처리
    for idx, row in enumerate(csv_reader):
        if idx == 0:
            continue

        # 3-7 열에 스토리
        story = ''
        story += ' '.join(row[2:6]) + ' '
        story = story.strip()
        story_dataset.append(story)

In [ ]:
len(story_dataset)

52665

In [ ]:
story_dataset[:10]

["David noticed he had put on a lot of weight recently. He examined his habits to try and figure out the reason. He realized he'd been eating too much fast food lately. He stopped going to burger places and started a vegetarian diet.",
 "Tom had a very short temper. One day a guest made him very angry. He punched a hole in the wall of his house. Tom's guest became afraid and left quickly.",
 'Marcus needed clothing for a business casual event. All of his clothes were either too formal or too casual. He decided to buy a pair of khakis. The pair he bought fit him perfectly.',
 'Bobby thought Bill should buy a trailer and haul it with his car. Bill thought a truck would be better for what he needed. Bobby pointed out two vehicles were much more expensive. Bill was set in his ways with conventional thinking.',
 'John was a pastor with a very bad memory. He tried to memorize his sermons many days in advance but to no avail. He decided to learn to sing to overcome his handicap. He then made 

### Preprocessing


In [ ]:
def extract_verb_in_stories(stories, make_one_string=True):
    # tokeniziong
    tokenizer = WordPunctTokenizer()

    story_tokens = []
    for story in stories:
        # stopword 제거
        tokens = tokenizer.tokenize(story)
        stop_words = list(set(stopwords.words('english')))
        tokens = [x for x in tokens if x not in stop_words]
        story_tokens.append(tokenizer.tokenize(story))

    # 동사만 추출 (원형으로)
    lemm = WordNetLemmatizer()
    story_verb_tokens = []

    # print('Extracting...')
    for story in story_tokens:
        verb_tokens = []
        for token, tag in pos_tag(story):
            if tag.startswith('V'):
                verb_tokens.append(lemm.lemmatize(token, pos='v'))
        story_verb_tokens.append(verb_tokens)

    if not make_one_string:
        return story_verb_tokens

    # 동사 토큰 이어서 하나의 string으로, tf-idf를 위해
    story_verb = []
    for tokens in story_verb_tokens:
            story_verb.append(' '.join(tokens))

    return story_verb

In [ ]:
story_verb = extract_verb_in_stories(story_dataset)

In [ ]:
for idx, ori in enumerate(story_dataset[:10]):
    print(f"{idx}: {ori} \n-> {story_verb[idx]}")

0: David noticed he had put on a lot of weight recently. He examined his habits to try and figure out the reason. He realized he'd been eating too much fast food lately. He stopped going to burger places and started a vegetarian diet. 
-> notice have put examine try figure realize be eat stop go burger start
1: Tom had a very short temper. One day a guest made him very angry. He punched a hole in the wall of his house. Tom's guest became afraid and left quickly. 
-> have make punch become leave
2: Marcus needed clothing for a business casual event. All of his clothes were either too formal or too casual. He decided to buy a pair of khakis. The pair he bought fit him perfectly. 
-> need be decide buy buy
3: Bobby thought Bill should buy a trailer and haul it with his car. Bill thought a truck would be better for what he needed. Bobby pointed out two vehicles were much more expensive. Bill was set in his ways with conventional thinking. 
-> think buy haul think be need point be be set
4:

In [ ]:
asdf = ["Ludo drove to his workplace in his car",
        "Ludo has completed a new project that needs to be completed urgently",
        "Ludo got the laptop from his company for work purposes",
        "Ludo was trying to impress his boss by working hard",
        "Ludo got the documents from his boss",
        "Ludo was working long hours without taking enough breaks",
        "Ludo's work was taking a toll on his health",
        "Ludo drove himself to hospital",
        "Ludo was not feeling well for a long time",
        "Ludo got a prescription for the medicine from his doctor",
        "Ludo was recovering from an illness",
        "Ludo drove to the bank",
        "Ludo applied for a credit card at his bank",
        "Ludo drove back home from his workplace",
        "Ludo purchased a subscription online using his credit card",
        "Ludo watched a lot of movies on the subscription during the next week."]

In [ ]:
for idx, ori in enumerate(asdf):
    print(f"{idx}: {ori} \n-> {extract_verb_in_stories(asdf)[idx]}")

0: Ludo drove to his workplace in his car 
-> drive
1: Ludo has completed a new project that needs to be completed urgently 
-> have complete need be complete
2: Ludo got the laptop from his company for work purposes 
-> get
3: Ludo was trying to impress his boss by working hard 
-> be try impress work
4: Ludo got the documents from his boss 
-> get
5: Ludo was working long hours without taking enough breaks 
-> be work take
6: Ludo's work was taking a toll on his health 
-> be take
7: Ludo drove himself to hospital 
-> drive hospital
8: Ludo was not feeling well for a long time 
-> be feel
9: Ludo got a prescription for the medicine from his doctor 
-> get
10: Ludo was recovering from an illness 
-> be recover
11: Ludo drove to the bank 
-> drive
12: Ludo applied for a credit card at his bank 
-> apply
13: Ludo drove back home from his workplace 
-> drive
14: Ludo purchased a subscription online using his credit card 
-> purchase use
15: Ludo watched a lot of movies on the subscriptio

### Train

In [ ]:
# TfidfVectorizer 객체 생성
vectorizer = TfidfVectorizer()

# vectorizer는 train에 등장하는 단어를 통해 단어 사전을 생성한 뒤,
# 해당 단어 사전을 사용해 train를 TfidfVectorizer 한다.
# 따라서, vec_train의 차원은 (X_train 문장 수, 사전의 단어 수) 가 된다.
vec_train = vectorizer.fit_transform(story_verb)

In [ ]:
# 벡터라이저가 학습한 단어사전 정렬
word_list = [i[0] for i in sorted(vectorizer.vocabulary_.items())]

# vec_train을 DataFrame 화
vec_train_df = pd.DataFrame(vec_train.toarray(), columns=[word_list])
vec_train_df

,aa,abandon,abduct,abide,ablaze,aboard,abolish,abort,absent,absinthe,...,youtube,yowl,zaddyfrack,zaddyfracks,zap,zigzag,zip,ziplining,zone,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Test

In [ ]:
(extract_verb_in_stories(asdf))

['drive',
 'have complete need be complete',
 'get',
 'be try impress work',
 'get',
 'be work take',
 'be take',
 'drive hospital',
 'be feel',
 'get',
 'be recover',
 'drive',
 'apply',
 'drive',
 'purchase use',
 'watch']

In [ ]:
def get_event_driven_score(trained_vector, plot, debug=False, is_simple=False):
    total_word_list = [i[0] for i in sorted(trained_vector.vocabulary_.items())]

    # fabula 전체에서의 이벤트(동사)에 대해 계산
    vec_total = trained_vector.transform([' '.join(list(set(extract_verb_in_stories(plot))))])
    total_df = pd.DataFrame(vec_total.toarray(), columns=[total_word_list])

    # 각 이벤트 문장 내의 이벤트(동사)에 대해 계산
    vec_each = TfidfVectorizer()
    #t = vec_each.fit_transform([' '.join(extract_verb_in_stories(plot))])
    t = vec_each.fit_transform(extract_verb_in_stories(plot))
    each_word_list = [i[0] for i in sorted(vec_each.vocabulary_.items())]
    vec_each = vec_each.transform(extract_verb_in_stories(plot))
    # vec_each = trained_vector.transform(extract_verb_in_stories(plot))
    each_df = pd.DataFrame(t.toarray(), columns=[each_word_list])

    # fabula 전체에 대한 이벤트(동사)에 대한 tf-idf 값
    total_event_score_dic = {}
    for index, row in total_df.iterrows():
        for column_name, value in row.items():
            if float(value) > 0:
                total_event_score_dic[column_name[0]] = float(value)

    # 정렬된 전체에 대한 tf-idf 값 출력
    if debug:
        print('\n[ fabula total event score (tf-idf) ]')
        sorted_list = [(key, value) for key, value in sorted(total_event_score_dic.items(), key=lambda item: item[1])]
        for key, value in sorted_list:
            print(f"{key}: {value}")
        print('\n')

    # 각 이벤트 문장 내의 이벤트(동사)에 대한 tf-idf 값
    if debug: print('[ each event score (tf-idf) ]')
    final_score_dic = {}
    for index, row in each_df.iterrows():
        if debug: print(f"{index}: {plot[index]}")
        sum = 0
        for column_name, value in row.items():
            if float(value) > 0:
                if debug: print(f"{column_name[0]}: {float(value)}")
                # 해당 이벤트 문장에 있는 tf-idf 값과 fabula 전체에 대한 tf-idf 값을 곱하여 합산 ()
                total_score = total_event_score_dic.get(column_name[0]) if total_event_score_dic.get(column_name[0]) is not None else 0
                sum += total_score * float(value)
        if not is_simple: final_score_dic[plot[index]] = sum
        if debug: print('\n')

    if is_simple:
        final_score_dic = {}
        for index, row in each_df.iterrows():
            if debug: print(f"{index}: {plot[index]}")
            sum = 0
            for column_name, value in row.items():
                if float(value) > 0:
                    if debug: print(f"{column_name[0]}: {float(value)}")
                    sum += float(value)
            if is_simple: final_score_dic[plot[index]] = sum

    # 정렬된 event-driven score 출력
    sorted_list = [(key, value) for key, value in sorted(final_score_dic.items(), key=lambda item: item[1])]
    if debug:
        print('[ final score ]')
        for key, value in sorted_list:
            print(f"{key}: {value}")

    return sorted_list

In [ ]:
qq = get_event_driven_score(vectorizer, asdf, debug=True)
qq


[ fabula total event score (tf-idf) ]
have: 0.07468498504916822
get: 0.09100691945360422
try: 0.1280410974734696
need: 0.14956047600604325
use: 0.16288298143313362
watch: 0.16915764555570809
feel: 0.18888516587765983
purchase: 0.22074352887039372
apply: 0.22773217357565723
take: 0.23227470217567564
impress: 0.2562540639508947
work: 0.28134798030054475
be: 0.2849015648284037
recover: 0.3025679854801814
drive: 0.3228744678682923
complete: 0.5293637954505815


[ each event score (tf-idf) ]
0: Ludo drove to his workplace in his car
drive: 1.0


1: Ludo has completed a new project that needs to be completed urgently
be: 0.23830420394385737
complete: 0.7929737727410175
have: 0.3964868863705088
need: 0.3964868863705088


2: Ludo got the laptop from his company for work purposes
get: 1.0


3: Ludo was trying to impress his boss by working hard
be: 0.3402895395701499
impress: 0.566168526512478
try: 0.566168526512478
work: 0.4930612826338269


4: Ludo got the documents from his boss
get: 1.0




[('Ludo got the laptop from his company for work purposes',
  0.09100691945360422),
 ('Ludo got the documents from his boss', 0.09100691945360422),
 ('Ludo got a prescription for the medicine from his doctor',
  0.09100691945360422),
 ('Ludo watched a lot of movies on the subscription during the next week.',
  0.16915764555570809),
 ('Ludo drove himself to hospital', 0.1866025607580151),
 ('Ludo applied for a credit card at his bank', 0.22773217357565723),
 ('Ludo purchased a subscription online using his credit card',
  0.27126490687855515),
 ('Ludo was not feeling well for a long time', 0.30866062681605605),
 ('Ludo drove to his workplace in his car', 0.3228744678682923),
 ('Ludo drove to the bank', 0.3228744678682923),
 ('Ludo drove back home from his workplace', 0.3228744678682923),
 ("Ludo's work was taking a toll on his health", 0.3529943756042941),
 ('Ludo was recovering from an illness', 0.4060981350734002),
 ('Ludo was working long hours without taking enough breaks',
  0.4513

In [ ]:
event_driven_score = get_event_driven_score(vectorizer, plot, debug=False)
event_driven_score

[('James loves sports', 0.06794304094611844),
 ('James took the school bus to get home', 0.07283019860430608),
 ('they had an important appointment', 0.07427058772160816),
 ('James walked to school', 0.07555019037452203),
 ('James purchased the sports equipment from a store', 0.10975935533864388),
 ('it was vital for the success of his project', 0.11805031632016916),
 ('it is a convenient', 0.11805031632016916),
 ('James owns a car for transportation convenience', 0.11950037882088968),
 ('James owns a car', 0.11950037882088968),
 ('James collected the various materials from different sources',
  0.12122992105528921),
 ('James bought the smartphone from a store', 0.12616837497530736),
 ('James does things that are convenient to make his life easier',
  0.13793149067153176),
 ('James went to different locations to gather the various materials',
  0.1444832013549904),
 ('James carried his smartphone with him wherever he went',
  0.1446579498125652),
 ("It was a crucial business meeting th

In [ ]:
event_driven_score_simple = get_event_driven_score(vectorizer, plot, debug=False, is_simple=True)
event_driven_score_simple

[('James owns a car for transportation convenience', 1.0),
 ('it was vital for the success of his project', 1.0),
 ('James collected the various materials from different sources', 1.0),
 ('James brought all the necessary materials', 1.0),
 ('James bought the smartphone from a store', 1.0),
 ('James received the appointment details via email', 1.0),
 ('they had an important appointment', 1.0),
 ('James drove his car to the current location from another place', 1.0),
 ('James drove his car to the current location', 1.0),
 ('James owns a car', 1.0),
 ('James drove his car to the store', 1.0),
 ('James purchased the sports equipment from a store', 1.0),
 ('James loves sports', 1.0),
 ('it is a convenient', 1.0),
 ('James walked to school', 1.0),
 ('James went to different locations to gather the various materials',
  1.414213562373095),
 ('James brought the necessary materials with him when he traveled to the location',
  1.414213562373095),
 ('James prepared the business documents himself

## graph-driven

https://jxnjxn.tistory.com/73
그래프 랜덤워크

https://eyeballs.tistory.com/36
PageRank

https://brain-nim.tistory.com/72
networkx PageRank

In [ ]:
def get_graph_driven_score(adj_list, goal, debug=False, is_weight=True):
    # NetworkX 그래프 생성
    plan_graph = nx.DiGraph()

    # 각 노드 추가
    for node in adj_list:
        plan_graph.add_node(node)

    # 각 엣지(링크) 추가
    for node, neighbors in adj_list.items():
        for neighbor in neighbors:
            if is_weight: plan_graph.add_edge(node, neighbor, weight=1/math.sqrt(search_path_in_adj_list(adj_list, node, goal)[0]))
            else: plan_graph.add_edge(node, neighbor)

    # 각 노드에 goal과의 거리만큼 가중치 부여
    nst = {}
    for node in plan_graph.nodes():
        nst[node] = search_path_in_adj_list(adj_list, node, goal)[0]
    nst = {key: math.sqrt(max(nst.values()) - value + 1) for key, value in nst.items()}

    # PageRank
    if is_weight: pagerank = nx.pagerank(plan_graph, nstart=nst)
    else: pagerank = nx.pagerank(plan_graph)

    # 정렬된 pagerank 출력
    if debug:
        sorted_list = [(key, value) for key, value in sorted(pagerank.items(), key=lambda item: item[1])]
        for key, value in sorted_list:
            print(f"{key}: {value}")

    return pagerank

In [ ]:
pagerank_w = get_graph_driven_score(adj_list, plot[-1], debug=False)
pagerank_w

{'James was glad to be playing a sport at school.': 0.03774479039136557,
 '(location) James is at school': 0.016530761535830087,
 '(reason) James loves sports': 0.024476857993048093,
 'James walked to school': 0.01604707200534016,
 'James loves sports': 0.02539868659928777,
 '(item_need) school bag': 0.015478112390544063,
 'James packed his school bag before leaving home': 0.014808820449353043,
 '(item_need) sports equipment': 0.026484356866818923,
 'James purchased the sports equipment from a store': 0.02776190779321567,
 '(location) James is at home': 0.008674484954094534,
 '(reason) James packed his school bag before leaving home so that he would have all his necessary materials for the day at school': 0.005346997626453751,
 'James took the school bus to get home': 0.006804780778961309,
 'James packed his school bag before leaving home so that he would have all his necessary materials for the day at school': 0.0028902478762693052,
 '(location) James is at the store': 0.0292646594389

In [ ]:
pagerank = get_graph_driven_score(adj_list, plot[-1], debug=False, is_weight=False)
pagerank

{'James was glad to be playing a sport at school.': 0.03774499192306364,
 '(location) James is at school': 0.016528877520263913,
 '(reason) James loves sports': 0.02448133442633587,
 'James walked to school': 0.016045442206659672,
 'James loves sports': 0.025406184376819033,
 '(item_need) school bag': 0.015476706543896162,
 'James packed his school bag before leaving home': 0.014807623032919926,
 '(item_need) sports equipment': 0.026493740467607843,
 'James purchased the sports equipment from a store': 0.027773363786700186,
 '(location) James is at home': 0.00867381801488364,
 '(reason) James packed his school bag before leaving home so that he would have all his necessary materials for the day at school': 0.005346749929646316,
 'James took the school bus to get home': 0.006804318259036055,
 'James packed his school bag before leaving home so that he would have all his necessary materials for the day at school': 0.0028901622107963747,
 '(location) James is at the store': 0.029275580395

In [ ]:
for i, key in enumerate(pagerank):
    if pagerank[key] != pagerank_w[key]:
        print(i, key)
        print(pagerank[key])
        print(pagerank_w[key])

0 James was glad to be playing a sport at school.
0.03774499192306364
0.03774479039136557
1 (location) James is at school
0.016528877520263913
0.016530761535830087
2 (reason) James loves sports
0.02448133442633587
0.024476857993048093
3 James walked to school
0.016045442206659672
0.01604707200534016
4 James loves sports
0.025406184376819033
0.02539868659928777
5 (item_need) school bag
0.015476706543896162
0.015478112390544063
6 James packed his school bag before leaving home
0.014807623032919926
0.014808820449353043
7 (item_need) sports equipment
0.026493740467607843
0.026484356866818923
8 James purchased the sports equipment from a store
0.027773363786700186
0.02776190779321567
9 (location) James is at home
0.00867381801488364
0.008674484954094534
10 (reason) James packed his school bag before leaving home so that he would have all his necessary materials for the day at school
0.005346749929646316
0.005346997626453751
11 James took the school bus to get home
0.006804318259036055
0.006

## main

In [ ]:
def skeleton_selection(plot, adj_list, vectorizer, k=5, a=0.5, debug=False, is_simple=False, is_weight=True):
    except_list = ['None', '(reason) None', '(location) None', '(item_need) None']

    # fabula
    fabula = [x for x in plot if x.strip() not in except_list][:-1]
    goal = plot[-1]

    # seletion score 초기화
    selection_score = {}
    for key, value in zip(fabula, [0]*len(fabula)):
        selection_score[key] = value

    # event-driven score
    event_driven_score = dict(get_event_driven_score(vectorizer, plot, debug=False, is_simple=is_simple))

    # 0 ~ a 로 정규화 (0 <= a <= 1)
    event_driven_score = {key: (value / max(event_driven_score.values())) * a for key, value in event_driven_score.items()}

    # selection score에 event-driven score 더하기
    if debug: print('[ event-driven ]')
    for key, value in selection_score.items():
        eds = event_driven_score.get(key) if event_driven_score.get(key) is not None else 0
        selection_score[key] += eds
        if debug: print(key, eds)

    # graph-driven score
    graph_driven_score = get_graph_driven_score(adj_list, goal, debug=False, is_weight=is_weight)

    # 0 ~ b 로 정규화 (b = 1-a)
    b = 1 - a
    graph_driven_score = {key: (value / max(graph_driven_score.values())) * b for key, value in graph_driven_score.items()}

    # selection score에 graph-driven score 더하기
    if debug: print('[ graph-driven ]')
    for key, value in selection_score.items():
        gds = graph_driven_score.get(key) if graph_driven_score.get(key) is not None else 0
        selection_score[key] += gds
        if debug: print(key, gds)

    # selection score 높은 이벤트 선택 (top k)
    top_k_selection = sorted(selection_score.items(), key=lambda item: item[1], reverse=True)[:k]   # [(node, score),...]
    top_k_selection = [x[0] for x in top_k_selection]

    # fabula에서 선택된 것 추출 (skeleton)
    skeleton = [x for x in fabula if x in top_k_selection] + [goal]

    return skeleton

In [ ]:
skeleton = skeleton_selection(plot, adj_list, vectorizer, k=9, a=0.5, debug=True)
skeleton

[ event-driven ]
Billy was on his way to an important meeting 0.1425334712227823
Billy is friends with the host of the special gathering 0.1425334712227823
it was the end of his workday 0.1425334712227823
Billy drove his car from work to reach his home 0.39170394175145784
Billy arrived at his home after a long day at work 0.11270646707468367
Billy received the invitation in the mailbox 0.13040818690631986
Billy was invited for a special gathering at his friend's house 0.1475306507577929
Billy got into his car to drive to a friend's house 0.44896758680444665
Billy was driving his car when the engine suddenly malfunctioned 0.4081397253431105
Billy is responsible for completing tasks assigned to him by his employer 0.24901144881628434
Billy commutes to work by driving his car for convenience 0.4208294848938301
Billy commutes to work by driving his car 0.4208294848938301
Billy received tasks from his work 0.13040818690631986
Billy had too many tasks 0.0448370025922087
Billy purchased the p

['Billy drove his car from work to reach his home',
 "Billy got into his car to drive to a friend's house",
 'Billy was driving his car when the engine suddenly malfunctioned',
 'Billy commutes to work by driving his car',
 'Billy drove his car to the store where he purchased the phone',
 'Billy uses a digital calendar app on his phone',
 'Billy managed to rearrange his schedule',
 "Billy didn't have enough time",
 'Billy neglected to maintain his car properly',
 'Billy totally blew his engine and was left on the side of the road.']

# Evaluation (A/B Test with GPT-3.5)

In [ ]:
class ABTester():
    def __init__(self, model='gpt-3.5-turbo-0613', max_tokens=1024, temperature=1.0, top_p=1, n=1, presence_penalty=1.0, frequency_penalty=1.0, api_key=None):
        openai.api_key = api_key
        self.model = model
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.top_p = top_p
        self.n = n
        self.presence_penalty = presence_penalty
        self.frequency_penalty = frequency_penalty
        self.skeleton = None
        self.gpt_skeleton = ''
        self.gpt_discourse = None

    def make_gpt_skeleton(self, adj_list, plot, debug=0):
        system_content = 'You create a skeleton story by selecting events from the tree-structured story planner. You have to look at the story planner given an adjacency list and choose 9 events in event list. The criteria for selecting events can be freely defined. Please select an appropriate event considering the fun of the event, causal rink, goal sentence, etc.'

        prompt = f'goal: {plot[-1]}'
        prompt += '\n\nadjacency list'
        for fr, to in adj_list.items():
            prompt += f'\n{fr} : {to}'
        prompt += '\n\nevent list'
        for event in plot[:-1]:
            prompt += f'\n{event}'
        prompt += f'\n\nQuestion: Choose 9 events in event list.\nAnswer: '

        if debug > 0:
            print(f'prompt:\n{prompt}')

        received = False
        while not received:
            try:
                self.gpt_skeleton = openai.ChatCompletion.create(
                    model=self.model,
                    max_tokens=self.max_tokens,
                    temperature=self.temperature,
                    top_p=self.top_p,
                    n=self.n,
                    presence_penalty=self.presence_penalty,
                    frequency_penalty=self.frequency_penalty,
                    messages = [{"role": "system", "content": system_content}, {"role": "user", "content": prompt}]
                )['choices'][0]['message']['content']
                received = True
            except Exception as e:
                if debug > 2:
                    print('Error in API:', e)
                time.sleep(1)

        self.gpt_skeleton += f'\n10. {plot[-1]}'

        if debug > 0:
            print(f'gpt_skeleton:\n{self.gpt_skeleton}')

    def reordering_gpt_skeleton(self, plot, debug=0):
        if self.gpt_skeleton == '': return
        # plot과 같은 순서로 reordering
        self.gpt_skeleton = self.gpt_skeleton.split('\n')
        self.gpt_skeleton = [s.split('.', 1)[1] if s.startswith(('1.', '2.', '3.', '4.', '5.', '6.', '7.', '8.', '9.', '10.')) else s for s in self.gpt_skeleton]
        self.gpt_skeleton = [s.strip() for s in self.gpt_skeleton]
        self.gpt_skeleton = [s[:-1] if s.endswith('.') else s for s in self.gpt_skeleton]
        tmp = []
        if debug > 0: tmp = self.gpt_skeleton
        self.gpt_skeleton = [x for x in plot[:-1] if x.lower() in [y.lower() for y in self.gpt_skeleton]]
        if debug > 0: print([x for x in tmp if x.lower() not in [y.lower() for y in self.gpt_skeleton]])
        self.gpt_skeleton.append(plot[-1])

    def get_gpt_skeleton(self):
        return self.gpt_skeleton

    def make_gpt_discourse(self, skeleton, debug=0):
        system_content = 'You are a story writer. Given the last sentence, S10, you complete the 10-sentence story [S1, S2, S3, S4, S5, S6, S7, S8, S9, S10]. The story should follow a single topic and be logically coherent. Each sentence should be under 20 words.'
        prompt = 'S10: ' + skeleton[-1]

        self.skeleton = skeleton

        received = False
        while not received:
            try:
                self.gpt_discourse = openai.ChatCompletion.create(
                    model=self.model,
                    max_tokens=self.max_tokens,
                    temperature=self.temperature,
                    top_p=self.top_p,
                    n=self.n,
                    presence_penalty=self.presence_penalty,
                    frequency_penalty=self.frequency_penalty,
                    messages = [{"role": "system", "content": system_content}, {"role": "user", "content": prompt}]
                )['choices'][0]['message']['content'] + '\nS10: ' + self.skeleton[-1]
                received = True
            except Exception as e:
                if debug > 2:
                    print('Error in API:', e)
                time.sleep(1)

        if debug > 0:
            print(self.gpt_discourse)

    def get_gpt_discourse(self):
        return self.gpt_discourse

    def test(self, question, repeat=10, debug=0):
        if self.skeleton is None:
            return -1

        system_content = 'You are the story evaluator. You just have to look at Story A and Story B, and answer the questions only with "A" or "B".'
        prompt = f"""Story A"""
        for i, s in enumerate(self.gpt_skeleton):
            prompt += f"""\n{i+1}. {s}"""
        prompt += f"""\n\nStory B"""
        for i, s in enumerate(self.skeleton):
            prompt += f"""\n{i+1}. {s}"""
        prompt += f"""\n\nQuestion: {question}\nAnswer: """

        if debug > 0:
            print(prompt)

        answer_list = []
        for i in range(repeat):
            received = False
            answer = ''
            i = 0
            while not received:
                i += 1
                try:
                    answer = openai.ChatCompletion.create(
                        model=self.model,
                        max_tokens=self.max_tokens,
                        temperature=self.temperature,
                        top_p=self.top_p,
                        n=self.n,
                        presence_penalty=self.presence_penalty,
                        frequency_penalty=self.frequency_penalty,
                        messages = [{"role": "system", "content": system_content}, {"role": "user", "content": prompt}],
                    )['choices'][0]['message']['content']
                    if answer == 'A' or answer == 'B' or i >= 20:
                        received = True
                except Exception as e:
                    if debug > 2:
                        print('Error in API:', e)
                    time.sleep(1)
            answer_list.append(answer)

        if debug > 0:
            print(answer_list)

        # 가장 많이 나온 값으로 리턴
        max_count = 0
        most_common_element = None

        for element in answer_list:
            count = answer_list.count(element)
            if count > max_count:
                max_count = count
                most_common_element = element

        return most_common_element, answer_list

In [ ]:
evaluation_model = ABTester(model='gpt-3.5-turbo-16k-0613', api_key=OPENAI_API_KEY)

In [ ]:
evaluation_model.skeleton

In [ ]:
evaluation_model.make_gpt_skeleton(adj_list, plot, debug=2)

prompt:
goal: Horace is glad the lightbulb in his bathroom is no longer dead.

adjacency list
Horace is glad the lightbulb in his bathroom is no longer dead. : set()
(item_need) a lightbulb : {'Horace is glad the lightbulb in his bathroom is no longer dead.'}
(location) Horace is in his bathroom : {'Horace is glad the lightbulb in his bathroom is no longer dead.', 'Horace had been using the lightbulb in his bathroom for a long time until it burned out', 'The old lightbulb burned out after being used for a long time'}
Horace bought a new lightbulb from a hardware store : {'(item_need) a lightbulb'}
None : {'(location) Horace is in his bathroom'}
(item_need) a new lightbulb : {'Horace bought a new lightbulb from a hardware store'}
(location) Horace is at the hardware store : {'Horace asked a store employee for assistance', 'Horace approached a store employee', 'Horace bought a new lightbulb from a hardware store'}
(reason) the old one in his bathroom was no longer working : {'Horace boug

In [ ]:
evaluation_model.gpt_skeleton = f"""1. Horace replaced the non-working lightbulb to have proper lighting in his bathroom.
2. Horace had been using the lightbulb in his bathroom for a long time until it burned out.
3. The old lightbulb burned out after being used for a long time.
4. Horace drove his car to the hardware store.
5. Horace bought a new lightbulb from a hardware store.
6. Horace took a taxi to the car dealership.
7. Horace bought a car from a dealership.
8. Horace walked to the street where he hailed the taxi.
9. Horace hailed a taxi on the street.
10. Horace is glad the lightbulb in his bathroom is no longer dead."""

In [ ]:
plot

['None',
 'Horace was transported to a location where he can freely move around by walking',
 'it is a natural',
 "Horace didn't have a choice in inheriting his functional legs",
 'Horace inherited his pair of functional legs from his parents',
 'Horace has had the ability to walk since he was born',
 'Horace walked to the street where he hailed the taxi',
 'Horace hailed a taxi on the street',
 'Horace took a taxi to the car dealership',
 'Horace bought a car from a dealership',
 'Horace drove his car to the hardware store',
 'Horace replaced the non-working lightbulb to have proper lighting in his bathroom',
 'Horace had been using the lightbulb in his bathroom for a long time until it burned out',
 'The old lightbulb burned out after being used for a long time',
 'the old one in his bathroom was no longer working',
 'he found that it brought him joy',
 'he has always had a natural desire to learn',
 'Horace developed a curious mindset through his innate curiosity',
 'Horace develope

In [ ]:
evaluation_model.reordering_gpt_skeleton(plot, debug=1)

['Horace is glad the lightbulb in his bathroom is no longer dead']


In [ ]:
for s in evaluation_model.get_gpt_skeleton():
    print(s)

Horace walked to the street where he hailed the taxi
Horace hailed a taxi on the street
Horace took a taxi to the car dealership
Horace bought a car from a dealership
Horace drove his car to the hardware store
Horace replaced the non-working lightbulb to have proper lighting in his bathroom
Horace had been using the lightbulb in his bathroom for a long time until it burned out
The old lightbulb burned out after being used for a long time
Horace bought a new lightbulb from a hardware store
Horace is glad the lightbulb in his bathroom is no longer dead.


In [ ]:
evaluation_model.test('Which story was more interesting?', debug=2) # Interestingness

Story A
1. Horace was transported to a location where he can freely move around by walking
2. Horace hailed a taxi on the street
3. Horace took a taxi to the car dealership
4. Horace bought a car from a dealership
5. Horace drove his car to the hardware store
6. Horace had been using the lightbulb in his bathroom for a long time until it burned out
7. The old lightbulb burned out after being used for a long time
8. Horace asked a store employee for assistance
9. Horace bought a new lightbulb from a hardware store
10. Horace is glad the lightbulb in his bathroom is no longer dead.

Story B
1. Horace didn't have a choice in inheriting his functional legs
2. Horace inherited his pair of functional legs from his parents
3. Horace has had the ability to walk since he was born
4. Horace walked to the street where he hailed the taxi
5. Horace hailed a taxi on the street
6. Horace had been using the lightbulb in his bathroom for a long time until it burned out
7. The old lightbulb burned out a

KeyboardInterrupt: ignored

In [ ]:
evaluation_model.test('Which story had coherent flow between sentences?', debug=2) # Logic Coherency

Story A
1. Horace was transported to a location where he can freely move around by walking
2. Horace hailed a taxi on the street
3. Horace took a taxi to the car dealership
4. Horace bought a car from a dealership
5. Horace drove his car to the hardware store
6. Horace had been using the lightbulb in his bathroom for a long time until it burned out
7. The old lightbulb burned out after being used for a long time
8. Horace asked a store employee for assistance
9. Horace bought a new lightbulb from a hardware store
10. Horace is glad the lightbulb in his bathroom is no longer dead.

Story B
1. Horace didn't have a choice in inheriting his functional legs
2. Horace inherited his pair of functional legs from his parents
3. Horace has had the ability to walk since he was born
4. Horace walked to the street where he hailed the taxi
5. Horace hailed a taxi on the street
6. Horace had been using the lightbulb in his bathroom for a long time until it burned out
7. The old lightbulb burned out a

KeyboardInterrupt: ignored

In [ ]:
evaluation_model.test('Which story had overall consistency in theme?', debug=2) # Topic Coherency

Story A
1. Horace was transported to a location where he can freely move around by walking
2. Horace hailed a taxi on the street
3. Horace took a taxi to the car dealership
4. Horace bought a car from a dealership
5. Horace drove his car to the hardware store
6. Horace had been using the lightbulb in his bathroom for a long time until it burned out
7. The old lightbulb burned out after being used for a long time
8. Horace asked a store employee for assistance
9. Horace bought a new lightbulb from a hardware store
10. Horace is glad the lightbulb in his bathroom is no longer dead.

Story B
1. Horace didn't have a choice in inheriting his functional legs
2. Horace inherited his pair of functional legs from his parents
3. Horace has had the ability to walk since he was born
4. Horace walked to the street where he hailed the taxi
5. Horace hailed a taxi on the street
6. Horace had been using the lightbulb in his bathroom for a long time until it burned out
7. The old lightbulb burned out a

KeyboardInterrupt: ignored

# Load & Selection & Evaluation using dataset

In [ ]:
# Load
loaded_path = 'results/fabula_/'
folder_names = []

for entry in os.listdir(loaded_path):
    full_path = os.path.join(loaded_path, entry)
    if os.path.isdir(full_path):
        folder_names.append(entry)
folder_names

['Wick met his death in front of the cathedral. (20)',
 'Laura quickly abandoned her corn garden idea. (17)',
 'Bill anonymously arranged to pay for all the funeral expenses. (23)',
 'Ludo watched a lot of movies on the subscription during the next week. (25)',
 'Tom sat on his couch filled with regret about his actions. (21)',
 'Anita melted the chocolate down and mixed it into the cake batter. (17)',
 'Bill buys his new dream car. (27)',
 'Kate ran in her new shoes. (22)',
 'Andy was excited to announce the opening of his landscaping business. (20)',
 'Anton closed the store for the day. (43)',
 'Bill calls triple a and waits for help. (22)',
 'Ann bought baby clothes at the department store. (18)',
 'Marcus was happy to have the right clothes for the event. (17)',
 'Annie had accidentally used salt instead of sugar! (18)',
 'Abby did not notice that Justin cheated to win the game. (29)',
 'Tom sat on his couch filled with regret about his actions. (46)',
 'Ben signed up and passed h

In [ ]:
len(folder_names)

135

In [ ]:
folder_names = folder_names[1:]
folder_names

['Bob sold off their belongings and used the money to buy a nice car. (65)',
 'Amy poured herself a large glass of Sherry and forgot about the flies. (50)',
 'Horace is glad the lightbulb in his bathroom is no longer dead. (26)']

In [ ]:
evaluation_model = ABTester(model='gpt-3.5-turbo-16k-0613', api_key=OPENAI_API_KEY)

In [ ]:
csv_filename = 'results/evaluation/evaluation_a0.5_6.csv'

if not os.path.isfile(csv_filename):
    open(csv_filename, 'w').close()

# 추가 데이터 생성
for folder_name in folder_names:
    new_evaluation = []

    plot, adj_list, network_data = load_plan(loaded_path + folder_name)
    skeleton = skeleton_selection(plot, adj_list, vectorizer, k=9, a=0.5)
    evaluation_model.make_gpt_skeleton(skeleton, adj_list, plot, debug=1)
    evaluation_model.reordering_gpt_skeleton(plot)
    gpt_skeleton = evaluation_model.get_gpt_skeleton()

    it, it_list = evaluation_model.test('Which story was more interesting?') # Interestingness
    lc, lc_list = evaluation_model.test('Which story had coherent flow between sentences?') # Logic Coherency
    tc, tc_list = evaluation_model.test('Which story had overall consistency in theme?') # Topic Coherency

    new_evaluation.append([folder_name, '\n'.join(gpt_skeleton), '\n'.join(skeleton), it, it_list, lc, lc_list, tc, tc_list])

    # CSV 파일 로드
    existing_data = []
    with open(csv_filename, 'r') as file:
        reader = csv.reader(file)
        existing_data = list(reader)

    # 기존 데이터와 추가 데이터 병합
    merged_data = existing_data + new_evaluation

    # CSV 파일 저장
    with open(csv_filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(merged_data)

prompt:
goal: Brenda is no longer in debt and now owns two homes.

adjacency list
Brenda is no longer in debt and now owns two homes. : set()
(item_need) two homes : {'Brenda is no longer in debt and now owns two homes.'}
(reason) Brenda worked hard : {'Brenda is no longer in debt and now owns two homes.'}
Brenda won the lottery : {'(item_need) two homes'}
Brenda worked hard : {'(reason) Brenda worked hard'}
(item_need) a winning lottery ticket : {'Brenda won the lottery'}
(location) Brenda is at the lottery office : {'Brenda won the lottery', 'Brenda purchased a lottery ticket'}
(reason) Brenda decided to try her luck : {'Brenda won the lottery'}
Brenda purchased a lottery ticket : {'(item_need) a winning lottery ticket'}
Brenda drove to the lottery office in her car : {'(location) Brenda is at the lottery office'}
Brenda decided to try her luck : {'(reason) Brenda decided to try her luck'}
(item_need) a strong work ethic : {'Brenda worked hard'}
Brenda was raised in a family that emp

KeyboardInterrupt: ignored

## add selection_list

In [ ]:
def skeletion_selection_with_path(path, a=0.5):
    plot, adj_list, network_data = load_plan(path)
    skeleton = skeleton_selection(plot, adj_list, vectorizer, k=9, a=a)
    return skeleton

In [ ]:
skeletion_selection_with_path('results/fabula_/James was glad to be playing a sport at school. (32)')

['James prepared the business documents himself by compiling to ensure accuracy',
 'James prepared the business documents himself by compiling',
 'James accessed his email using his smartphone to stay connected',
 'James accessed his email using his smartphone',
 'they had an important appointment',
 'James drove his car to the current location to meet someone',
 'James drove his car to the current location from another place',
 'James drove his car to the current location',
 'James drove his car to the store',
 'James was glad to be playing a sport at school.']

In [ ]:
selection_list_path = 'results/evaluation/selection_list.csv'
selection_name = 'ss-simple_0.5'

# CSV 파일 열림 (읽기 모드)
with open(selection_list_path, mode='r') as file:
    # CSV 파일을 읽기 위한 reader 생성
    reader = csv.reader(file)

    # 헤더(첫 번째 행)를 읽어옴
    header = next(reader)

    # 첫 번째 열의 각 값을 asdf() 함수를 적용하여 새로운 열에 추가
    header.append(selection_name)  # 새로운 열 이름

    # 데이터 행들을 수정하여 새로운 열에 추가
    data = []
    for row in tqdm(reader, desc=f'Selection({selection_name})', unit=' skeleton'):
        skeleton = '\n'.join(skeletion_selection_with_path('results/fabula_/'+row[0], a=0.5))
        data.append(row + [skeleton])

# 열 추가와 데이터 추가가 포함된 CSV 파일 열기 (쓰기 모드)
with open(selection_list_path, mode='w', newline='') as file:
    # CSV 파일을 쓰기 위한 writer 생성
    writer = csv.writer(file)

    # 새로운 헤더 쓰기
    writer.writerow(header)

    # 새로운 데이터 행들을 쓰기
    writer.writerows(data)

Selection(ss-simple_0.5): 135 skeleton [00:19,  6.98 skeleton/s]


## use selection_list

In [ ]:
evaluation_model = ABTester(model='gpt-3.5-turbo-16k-0613', api_key=OPENAI_API_KEY)

In [ ]:
selection_list_path = 'results/evaluation/selection_list.csv'
column_a = 'gpt_hu10'
column_b = 'ss_1'

evolution_path = f'results/evaluation/{column_a}_vs_{column_b}_2.csv'
if not os.path.isfile(evolution_path):
    open(evolution_path, 'w').close()

with open(selection_list_path, mode='r') as file:
    reader = csv.DictReader(file)

    # 입력으로 받은 열 이름에 해당하는 데이터 출력 및 진행 상황 표시
    total_rows = sum(1 for row in reader)
    file.seek(0)  # 파일을 처음으로 되돌림
    next(reader)  # 헤더 행을 건너뜁니다.

    for row in tqdm(reader, total=total_rows, desc="A/B Testing", unit=" fabula"):
        if column_a in row and column_b in row:
            skeleton_a = row[column_a].split('\n')
            skeleton_b = row[column_b].split('\n')

            new_evaluation = []

            evaluation_model.gpt_skeleton = skeleton_a
            evaluation_model.skeleton = skeleton_b

            it, it_list = evaluation_model.test('Which story was more interesting?') # Interestingness
            lc, lc_list = evaluation_model.test('Which story had coherent flow between sentences?') # Logic Coherency
            tc, tc_list = evaluation_model.test('Which story had overall consistency in theme?') # Topic Coherency

            new_evaluation.append([row['goal'], '\n'.join(skeleton_a), '\n'.join(skeleton_b), it, it_list, lc, lc_list, tc, tc_list])

            # CSV 파일 로드
            existing_data = []
            with open(evolution_path, 'r') as file:
                reader = csv.reader(file)
                existing_data = list(reader)

            # 기존 데이터와 추가 데이터 병합
            merged_data = existing_data + new_evaluation

            # CSV 파일 저장
            with open(evolution_path, 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerows(merged_data)

A/B Testing: 100%|██████████| 135/135 [46:26<00:00, 20.64s/ fabula]


# Visualization

In [14]:
def print_evaluation(path):
    evaluation = []
    with open(path, 'r') as file:
            reader = csv.reader(file)
            evaluation = list(reader)

    it = []
    lc = []
    tc = []
    for eval in evaluation:
        #it.append(eval[3])
        #lc.append(eval[5])
        #tc.append(eval[7])
        it = it + [char for char in eval[4]]
        lc = lc + [char for char in eval[6]]
        tc = tc + [char for char in eval[8]]

    print('Interestingness: A(' + str(len([x for x in it if x == 'A'])/len([a for a in it if a =='A' or a == 'B'])) + '), B(' + str(len([x for x in it if x == 'B'])/len([a for a in it if a =='A' or a == 'B'])) + ')')
    print('Logic Coherency: A(' + str(len([x for x in lc if x == 'A'])/len([a for a in lc if a =='A' or a == 'B'])) + '), B(' + str(len([x for x in lc if x == 'B'])/len([a for a in lc if a =='A' or a == 'B'])) + ')')
    print('Topic Coherency: A(' + str(len([x for x in tc if x == 'A'])/len([a for a in tc if a =='A' or a == 'B'])) + '), B(' + str(len([x for x in tc if x == 'B'])/len([a for a in tc if a =='A' or a == 'B'])) + ')')

In [12]:
print('\n[ a = 0 ]')
print_evaluation('results/evaluation/gpt_hu10_vs_ss_0.csv')

print('\n[ a = 0.25 ]')
print_evaluation('results/evaluation/gpt_hu10_vs_ss_0.25.csv')

print('\n[ a = 0.5 ]')
print_evaluation('results/evaluation/gpt_hu10_vs_ss_0.5_2.csv')

print('\n[ a = 0.75 ]')
print_evaluation('results/evaluation/gpt_hu10_vs_ss_0.75.csv')

print('\n[ a = 1 ]')
print_evaluation('results/evaluation/gpt_hu10_vs_ss_1.csv')


[ a = 0 ]
Interestingness: A(0.23703703703703705), B(0.762962962962963)
Logic Coherency: A(0.14074074074074075), B(0.8592592592592593)
Topic Coherency: A(0.14814814814814814), B(0.8518518518518519)

[ a = 0.25 ]
Interestingness: A(0.2740740740740741), B(0.725925925925926)
Logic Coherency: A(0.14814814814814814), B(0.8518518518518519)
Topic Coherency: A(0.15555555555555556), B(0.8444444444444444)

[ a = 0.5 ]
Interestingness: A(0.28888888888888886), B(0.7111111111111111)
Logic Coherency: A(0.21481481481481482), B(0.7851851851851852)
Topic Coherency: A(0.11851851851851852), B(0.8814814814814815)

[ a = 0.75 ]
Interestingness: A(0.23703703703703705), B(0.762962962962963)
Logic Coherency: A(0.26666666666666666), B(0.7333333333333333)
Topic Coherency: A(0.22962962962962963), B(0.7703703703703704)

[ a = 1 ]
Interestingness: A(0.23703703703703705), B(0.762962962962963)
Logic Coherency: A(0.14074074074074075), B(0.8592592592592593)
Topic Coherency: A(0.14814814814814814), B(0.851851851851851

In [15]:
print('\na = 0.5')

print('\n[ simple TF-IDF ]')
print_evaluation('results/evaluation/gpt_hu10_vs_ss-simple_0.5.csv')

print('\n[ no weight PageRank ]')
print_evaluation('results/evaluation/gpt_hu10_vs_ss-noweight_0.5.csv')


a = 0.5

[ simple TF-IDF ]
Interestingness: A(0.36666666666666664), B(0.6333333333333333)
Logic Coherency: A(0.30518518518518517), B(0.6948148148148148)
Topic Coherency: A(0.21555555555555556), B(0.7844444444444445)

[ no weight PageRank ]
Interestingness: A(0.3909495548961424), B(0.6090504451038575)
Logic Coherency: A(0.2985185185185185), B(0.7014814814814815)
Topic Coherency: A(0.23703703703703705), B(0.762962962962963)


# Evaluation (legacy)


In [ ]:
import time

plot = f"""
He loves wearing them.
Casper is happy he got a new pair of shorts.
Casper wants a new pair of shorts for his daily runs.
He decides to buy a nice new pair.
Casper finds the perfect pair of shorts.
"""

plot = [x for x in plot.split("\n")]
plot = [x for x in plot if x]

total_num = 0
cnt = 0
for i in range(len(plot) - 1):
    for j in range(i, len(plot)):
        p_action = ' '.join(plot[i:j])
        q_action = plot[j]

        message = f"""
        Can '{p_action}' enable '{q_action}'? Just answer 'yes' or 'no'
        """

        messages = [{"role": "system", "content": "You are a story coherence evaluator." }, {"role": "user", "content": message}]

        try:
            answer = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            api_key=OPENAI_API_KEY,
            messages=messages
            )['choices'][0]['message']['content']

            total_num += 1

            answer = answer.lower()

            if 'yes' in answer:
                cnt += 1

            print(answer)

        except Exception as e:
            print("Error in API: ", e)
            time.sleep(1)

print(cnt, total_num)
if cnt >= total_num * 0.7 and total_num :
    print("Good Plot")
elif not total_num:
    print("API Error")
else:
    print("Bad Plot")

In [ ]:
"""
John his patient has heart pain.
John was taken to get to hospital.
John got stethoscope from medical kit.
John diagnosed heart pain in his patient at hospital
"""

q_action = 'John diagnosed heart pain in his patient at hospital'

q_prompt = f"""
John his patient has heart pain.
John was taken to get to hospital.
John got stethoscope from medical kit.

If the context has an action that might enable the '{q_action}', select it. If not, answer none.
"""

qa_model =  GPT('gpt-3.5-turbo', api_key=OPENAI_API_KEY, temperature=0.7)
qa = qa_model.generate(q_prompt)
print(qa)

In [ ]:
message ="""
Evaluate the story
"Adam likes scary things
Adam is sitting in front of a TV
Adam orders a video game online
Adam started playing the zombie video game
Adam is playing a zombie video game
Adam was trying to defend himself against zombies
Adam rode in his car to find a zombie infested area
Adam drove his car to the area where he was hunting zombies with a bat
Adam hunts zombies with a bat"
's coherence in scale of 1 to 100
"""
messages = [{"role": "system", "content": "You are a story coherence evaluator." }, {"role": "user", "content": message}]

# qa_model =  GPT('gpt-3.5-turbo', api_key=OPENAI_API_KEY, temperature=0.7)
# qa = qa_model.create(messages)
# print(qa)

answer = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages
)['choices'][0]['message']['content']

print(answer)

# Note

추가 실험
- graph-driven 골과의 거리 가중치 주고 결과 (a = [0, 0.25, 0.5, 0.75, 1])
- a = 1로 해두고 story 요약 벤치마크와 비교
- ablation stody (일반 idf로 해보기, 일반 pagerank로 해보기)

reference 대략 20~30

t-test 할거면 갯수로 (안나오면 배제) 통계적으로 유의하지는 않았지만~~~ (분위기상 안써도될듯)

arr 재제출 되는지 찾아보기 --> Paper submission deadline (via ARR)	Sunday	15 October 2023 까지 재제출 또는 수정 가능
https://aclrollingreview.org/authors (ARR에 대한 공식 안내 링크)

.

평가 신뢰성 (gpt, 아마존)
요약 벤치마크와 비교

오더링 --> 플래시백, 플래시포워드

스켈레톤 요약 다른 경우 --> 중간 채워서 자연스러운 스토리로